In [3]:
import pandas as pd

In [4]:
! ls

6505_MLP_predicted_movement.csv
best_lg_absa.csv
best_processed_day_level_absa.csv
data preparation.ipynb
feature engineering for day-level data.ipynb
lg_absa.csv
lg_sa_bert.csv
lg_sa_roberta.csv
processed.zip
processed_absa_price_diff.csv
processed_day_level_absa.csv
processed_day_level_sa_bert.csv
processed_day_level_sa_roberta.csv
processed_sa_bert_price_diff.csv
processed_sa_roberta_price_diff.csv
sample.py


# load stock price

In [5]:
def load_stock_df():
    import yfinance as yf

    # Define the stock ticker and date range
    ticker = "LPL"  # Use the correct ticker symbol for LG
    start_date = "2018-01-01"
    end_date = "2018-12-31"

    # Fetch the stock data
    stock_data = yf.download(ticker, start=start_date, end=end_date)

    # Extract the adjusted close prices
    adjusted_close_prices = stock_data["Adj Close"]

    print(adjusted_close_prices)
    return adjusted_close_prices

In [6]:
lg_stock = load_stock_df()
lg_stock

[*********************100%***********************]  1 of 1 completed
Date
2018-01-02    14.11
2018-01-03    14.48
2018-01-04    14.29
2018-01-05    14.25
2018-01-08    14.26
              ...  
2018-12-21     7.91
2018-12-24     7.99
2018-12-26     8.22
2018-12-27     8.41
2018-12-28     8.13
Name: Adj Close, Length: 250, dtype: float64


Date
2018-01-02    14.11
2018-01-03    14.48
2018-01-04    14.29
2018-01-05    14.25
2018-01-08    14.26
              ...  
2018-12-21     7.91
2018-12-24     7.99
2018-12-26     8.22
2018-12-27     8.41
2018-12-28     8.13
Name: Adj Close, Length: 250, dtype: float64

In [7]:
known_movement_df = lg_stock.diff() # 已知信息：今天的所有情绪信息 + 昨天收盘到今天收盘的价差
print(known_movement_df) # 预测：今天收盘到明天收盘的价差，即：明天的价格movement
to_predict_movement = known_movement_df.shift(-1)
print(to_predict_movement)

Date
2018-01-02     NaN
2018-01-03    0.37
2018-01-04   -0.19
2018-01-05   -0.04
2018-01-08    0.01
              ... 
2018-12-21   -0.02
2018-12-24    0.08
2018-12-26    0.23
2018-12-27    0.19
2018-12-28   -0.28
Name: Adj Close, Length: 250, dtype: float64
Date
2018-01-02    0.37
2018-01-03   -0.19
2018-01-04   -0.04
2018-01-05    0.01
2018-01-08   -0.14
              ... 
2018-12-21    0.08
2018-12-24    0.23
2018-12-26    0.19
2018-12-27   -0.28
2018-12-28     NaN
Name: Adj Close, Length: 250, dtype: float64


In [95]:
type(known_movement_df)

pandas.core.series.Series

In [11]:
# Convert the two series into dataframes
known_movement_df = known_movement_df.to_frame()
known_movement_df.columns = ['known']
to_predict_movement = to_predict_movement.to_frame()
to_predict_movement.columns = ['to_predict']

In [97]:
type(known_movement_df)

pandas.core.frame.DataFrame

# sentiment analysis

In [86]:
df = pd.read_csv('./processed_day_level_sa_bert.csv', index_col=[0])
df.index.name = 'Date'
df

,positive,negative,neutral,none,count
Date,,,,,
2018-01-01,10,2,1,0,13
2018-01-02,28,7,1,0,36
2018-01-03,22,11,1,0,34
2018-01-04,24,8,4,0,36
2018-01-05,21,5,2,0,28
...,...,...,...,...,...
2018-05-27,12,2,0,0,14
2018-05-28,17,7,3,0,27
2018-05-29,20,5,2,0,27


In [90]:
# Convert the index of the df dataframe to DatetimeIndex
df.index = pd.to_datetime(df.index, format='%Y-%m-%d')

# Concatenate the three dataframes on their index
full = pd.concat([df, known_movement_df, to_predict_movement], axis=1, join='inner')

# Set the name of the index column
full.index.name = 'Date'

# Drop any rows with missing values
full.dropna(inplace=True)

In [91]:
full

,positive,negative,neutral,none,count,known,to_predict
Date,,,,,,,
2018-01-03,22,11,1,0,34,0.37,-0.190000
2018-01-04,24,8,4,0,36,-0.19,-0.040000
2018-01-05,21,5,2,0,28,-0.04,0.010000
2018-01-08,24,1,1,0,26,0.01,-0.140000
2018-01-09,25,8,5,0,38,-0.14,-0.140000
...,...,...,...,...,...,...,...
2018-05-24,14,5,3,0,22,-0.17,-0.040000
2018-05-25,14,3,1,0,18,-0.04,0.120000
2018-05-29,20,5,2,0,27,0.12,0.010000


In [92]:
full.to_csv('processed_sa_bert_price_diff.csv')

## for roberta

In [98]:
df = pd.read_csv('./processed_day_level_sa_roberta.csv', index_col=[0])
df.index.name = 'Date'
df

,positive,negative,neutral,none,count
Date,,,,,
2018-01-01,11,1,1,0,13
2018-01-02,27,7,2,0,36
2018-01-03,24,8,2,0,34
2018-01-04,25,7,4,0,36
2018-01-05,21,6,1,0,28
...,...,...,...,...,...
2018-05-27,12,1,1,0,14
2018-05-28,17,7,3,0,27
2018-05-29,21,4,2,0,27


In [99]:
# Convert the index of the df dataframe to DatetimeIndex
df.index = pd.to_datetime(df.index, format='%Y-%m-%d')

# Concatenate the three dataframes on their index
full = pd.concat([df, known_movement_df, to_predict_movement], axis=1, join='inner')

# Set the name of the index column
full.index.name = 'Date'

# Drop any rows with missing values
full.dropna(inplace=True)

In [100]:
full

,positive,negative,neutral,none,count,known,to_predict
Date,,,,,,,
2018-01-03,24,8,2,0,34,0.37,-0.190000
2018-01-04,25,7,4,0,36,-0.19,-0.040000
2018-01-05,21,6,1,0,28,-0.04,0.010000
2018-01-08,23,2,1,0,26,0.01,-0.140000
2018-01-09,27,6,5,0,38,-0.14,-0.140000
...,...,...,...,...,...,...,...
2018-05-24,15,6,1,0,22,-0.17,-0.040000
2018-05-25,15,1,2,0,18,-0.04,0.120000
2018-05-29,21,4,2,0,27,0.12,0.010000


In [101]:
full.to_csv('processed_sa_roberta_price_diff.csv')

# absa feature engineering

In [108]:
df = pd.read_csv('./processed_day_level_absa.csv', index_col=[0])
df.index.name = 'Date'
df

,quality_positive,quality_negative,quality_neutral,quality_none,connectivity_positive,connectivity_negative,connectivity_neutral,connectivity_none,usability_positive,usability_negative,...,operation performance_none,design features_positive,design features_negative,design features_neutral,design features_none,price_positive,price_negative,price_neutral,price_none,count
Date,,,,,,,,,,,,,,,,,,,,,
2018-01-01,4,0,0,9,0,1,0,12,0,0,...,9,1,0,0,12,0,0,0,13,13
2018-01-02,10,2,0,24,0,1,0,35,0,0,...,23,7,1,0,28,4,1,0,31,36
2018-01-03,5,4,0,25,0,0,0,34,2,0,...,25,9,1,0,24,3,0,0,31,34
2018-01-04,9,4,0,23,0,0,0,36,2,0,...,27,6,3,0,27,2,0,0,34,36
2018-01-05,8,1,0,19,0,0,0,28,1,0,...,22,5,0,0,23,3,0,0,25,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-05-27,6,1,0,7,0,0,0,14,3,0,...,8,5,0,0,9,3,0,0,11,14
2018-05-28,7,5,0,15,0,0,0,27,1,0,...,19,1,2,0,24,1,0,0,26,27
2018-05-29,6,1,0,20,0,0,0,27,1,0,...,20,4,1,0,22,2,1,0,24,27


In [109]:
# Convert the index of the df dataframe to DatetimeIndex
df.index = pd.to_datetime(df.index, format='%Y-%m-%d')

# Concatenate the three dataframes on their index
full = pd.concat([df, known_movement_df, to_predict_movement], axis=1, join='inner')

# Set the name of the index column
full.index.name = 'Date'

# Drop any rows with missing values
full.dropna(inplace=True)

In [110]:
full

,quality_positive,quality_negative,quality_neutral,quality_none,connectivity_positive,connectivity_negative,connectivity_neutral,connectivity_none,usability_positive,usability_negative,...,design features_negative,design features_neutral,design features_none,price_positive,price_negative,price_neutral,price_none,count,known,to_predict
Date,,,,,,,,,,,,,,,,,,,,,
2018-01-03,5,4,0,25,0,0,0,34,2,0,...,1,0,24,3,0,0,31,34,0.37,-0.190000
2018-01-04,9,4,0,23,0,0,0,36,2,0,...,3,0,27,2,0,0,34,36,-0.19,-0.040000
2018-01-05,8,1,0,19,0,0,0,28,1,0,...,0,0,23,3,0,0,25,28,-0.04,0.010000
2018-01-08,8,0,0,18,0,0,0,26,1,0,...,0,0,19,3,0,0,23,26,0.01,-0.140000
2018-01-09,8,3,0,27,0,0,0,38,2,0,...,3,0,28,4,0,0,34,38,-0.14,-0.140000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-05-24,7,3,0,12,0,0,0,22,1,0,...,0,0,19,5,0,0,17,22,-0.17,-0.040000
2018-05-25,5,0,0,13,0,0,0,18,1,0,...,0,0,17,5,0,0,13,18,-0.04,0.120000
2018-05-29,6,1,0,20,0,0,0,27,1,0,...,1,0,22,2,1,0,24,27,0.12,0.010000


In [111]:
full.to_csv('processed_absa_price_diff.csv')

In [106]:
! ls

data preparation.ipynb
feature engineering for day-level data.ipynb
lg_absa.csv
lg_sa_bert.csv
lg_sa_roberta.csv
processed_absa_price_diff.csv
processed_day_level_absa.csv
processed_day_level_sa_bert.csv
processed_day_level_sa_roberta.csv
processed_sa_bert_price_diff.csv
processed_sa_roberta_price_diff.csv
sample.py


---

In [12]:
import pandas as pd
df = pd.read_csv('./best_processed_day_level_absa.csv', index_col=[0])
df.index.name = 'Date'

# Convert the index of the df dataframe to DatetimeIndex
df.index = pd.to_datetime(df.index, format='%Y-%m-%d')

# Concatenate the three dataframes on their index
full = pd.concat([df, known_movement_df, to_predict_movement], axis=1, join='inner')

# Set the name of the index column
full.index.name = 'Date'

# Drop any rows with missing values
full.dropna(inplace=True)

In [13]:
full

,quality_positive,quality_negative,quality_neutral,quality_none,connectivity_positive,connectivity_negative,connectivity_neutral,connectivity_none,usability_positive,usability_negative,...,design features_negative,design features_neutral,design features_none,price_positive,price_negative,price_neutral,price_none,count,known,to_predict
Date,,,,,,,,,,,,,,,,,,,,,
2018-01-03,4,4,0,26,0,0,0,34,1,0,...,0,0,28,3,0,0,31,34,0.37,-0.190000
2018-01-04,9,2,0,25,0,0,0,36,1,0,...,1,0,29,2,0,0,34,36,-0.19,-0.040000
2018-01-05,7,1,0,20,0,0,0,28,1,0,...,0,0,24,3,0,0,25,28,-0.04,0.010000
2018-01-08,10,0,0,16,0,0,0,26,1,0,...,0,0,22,3,0,0,23,26,0.01,-0.140000
2018-01-09,9,2,0,27,0,0,0,38,1,0,...,2,0,31,3,1,0,34,38,-0.14,-0.140000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-05-24,6,2,0,14,0,0,0,22,1,0,...,0,0,19,5,0,0,17,22,-0.17,-0.040000
2018-05-25,2,1,0,15,0,0,0,18,1,0,...,0,0,15,4,0,0,14,18,-0.04,0.120000
2018-05-29,5,0,0,22,0,0,0,27,2,0,...,1,1,23,2,1,0,24,27,0.12,0.010000


In [14]:
full.to_csv('best_processed_absa_price_diff.csv')